<a href="https://www.kaggle.com/code/chandlertimm/black-clover-manga-colorization?scriptVersionId=95809400" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import json
import cv2
import matplotlib
import matplotlib.pyplot as plt
import scipy
import warnings
import shutil
import random
import shutil 

from PIL import Image
from tqdm import tqdm

# Process the dataset

In [2]:
!cp -r ../input/black-clover-manga-dataset/colored ./
!cp -r ../input/black-clover-manga-dataset/grayscale ./

In [3]:
# colored
colored_files =  os.listdir('./colored')
sorted_colored =  sorted(colored_files, key=lambda x: int(os.path.splitext(x)[0]))

# grayscale
grayscale_files =  os.listdir('./grayscale')
sorted_grayscale =  sorted(grayscale_files, key=lambda x: int(os.path.splitext(x)[0]))

In [4]:
# shuffle the dataset
temp = list(zip(sorted_colored, sorted_grayscale))

random.shuffle(temp)
res1, res2 = zip(*temp)

# res1 and res2 come out as tuples, and so must be converted to lists.
colored, grayscale = list(res1), list(res2)

print(colored[0:5])
print(grayscale[0:5])

['201.png', '441.png', '466.png', '561.png', '616.png']
['201.png', '441.png', '466.png', '561.png', '616.png']


In [5]:
#split the dataset 0.8 for training

#training colored
train_cl = []
for i in tqdm(range(int(0.0*len(colored)), int(0.8*len(colored)))):
    train_cl.append(colored[i])
    
#testing grayscale
test_cl = []
for i in tqdm(range(int(0.8*len(colored)), int(1.0*len(colored)))):
    test_cl.append(colored[i])

100%|██████████| 126/126 [00:00<00:00, 76780.81it/s]


In [6]:
print(train_cl[-3:None])
print(test_cl[0:3])

['269.png', '341.png', '579.png']
['271.png', '114.png', '504.png']


In [7]:
#training grayscale
train_bw = []
for i in tqdm(range(int(0.0*len(grayscale)), int(0.8*len(grayscale)))):
    train_bw.append(grayscale[i])
    
#testing grayscale
test_bw = []
for i in tqdm(range(int(0.8*len(grayscale)), int(1.0*len(grayscale)))):
    test_bw.append(grayscale[i])

100%|██████████| 126/126 [00:00<00:00, 393508.79it/s]


In [8]:
print(train_bw[-3:None])
print(test_bw[0:3])

['269.png', '341.png', '579.png']
['271.png', '114.png', '504.png']


In [9]:
# create the dataset folders
if not os.path.isdir('./blackclover'):
    os.makedirs('./blackclover')
    # unpaired image dataset path
    os.makedirs('./blackclover/trainA/')
    os.makedirs('./blackclover/trainB/')
    os.makedirs('./blackclover/testA/')
    os.makedirs('./blackclover/testB/')

In [10]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [11]:
# copy the train dataset to unpaired image dataset path

for i in tqdm(range(len(train_cl)), 'copying train colored and grayscale'):
    # Source path 
    src1 = find(str(train_bw[i]), './grayscale/')
    src2 = find(str(train_cl[i]), './colored/')

    # Destination path 
    dest1 = './blackclover/trainA/'
    dest2 = './blackclover/trainB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2)  

copying train colored and grayscale: 100%|██████████| 504/504 [00:01<00:00, 291.24it/s]


In [12]:
# copy the test dataset to unpaired image dataset path

for i in tqdm(range(len(test_cl)), 'copying test colored and grayscale'):
    # Source path 
    src1 = find(str(test_bw[i]), './grayscale/')
    src2 = find(str(test_cl[i]), './colored/')

    # Destination path 
    dest1 = './blackclover/testA/'
    dest2 = './blackclover/testB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2) 

copying test colored and grayscale: 100%|██████████| 126/126 [00:00<00:00, 323.61it/s]


# Install MMGeneration

In [13]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

# Check Pytorch installation
import torch
print(torch.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

1.6.0


In [14]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 894 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


In [15]:
!conda install pytorch==1.6.0 torchvision==0.7.0 cudatoolkit=10.1 -c pytorch -y

# install the latest mmcv
# pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.6.0/index.html

# install mmgeneration
!rm -rf mmgeneration
!git clone https://github.com/open-mmlab/mmgeneration.git
%cd mmgeneration
!pip install -r requirements.txt
!pip install -v -e .

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit=10.1
    - pytorch==1.6.0
    - torchvision==0.7.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_2         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    -------------------------------------------------------

In [16]:
from mmcv import collect_env
collect_env()

{'sys.platform': 'linux',
 'Python': '3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) [GCC 7.3.0]',
 'CUDA available': True,
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'CUDA_HOME': '/usr/local/cuda',
 'NVCC': 'Cuda compilation tools, release 10.1, V10.1.24',
 'GCC': 'gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'PyTorch': '1.6.0',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v1.5.0 (Git Hash e2ac1fac44c5078ca927cb9b90e1b3066a0b2ed0)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 10.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;

In [17]:
# Check MMDetection installation
import mmgen
print(mmgen.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

0.7.1
10.1
GCC 7.3


# Start training

In [18]:
!cp -r ../../input/black-clover-colorization-output/mmgeneration/work_dirs ./

In [19]:
%%writefile configs/cyclegan/cyclegan_lsgan_id0_resnet_in_summer2winter_b1x1_250k.py

_base_ = [
    '../_base_/models/cyclegan/cyclegan_lsgan_resnet.py',
    '../_base_/datasets/unpaired_imgs_256x256.py',
    '../_base_/default_runtime.py'
]
domain_a = 'grayscale'
domain_b = 'colored'
model = dict(
    default_domain=domain_b,
    reachable_domains=[domain_a, domain_b],
    related_domains=[domain_a, domain_b],
    gen_auxiliary_loss=[
        dict(
            type='L1Loss',
            loss_weight=10.0,
            loss_name='cycle_loss',
            data_info=dict(
                pred=f'cycle_{domain_a}', target=f'real_{domain_a}'),
            reduction='mean'),
        dict(
            type='L1Loss',
            loss_weight=10.0,
            loss_name='cycle_loss',
            data_info=dict(
                pred=f'cycle_{domain_b}',
                target=f'real_{domain_b}',
            ),
            reduction='mean')
    ])
dataroot = '.././blackclover/'
train_pipeline = [
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_a}',
        flag='color'),
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_b}',
        flag='color'),
    dict(
        type='Resize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        scale=(512, 512),
        interpolation='bicubic'),
    dict(
        type='Crop',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        crop_size=(256, 256),
        random_crop=True),
    dict(type='Flip', keys=[f'img_{domain_a}'], direction='horizontal'),
    dict(type='Flip', keys=[f'img_{domain_b}'], direction='horizontal'),
    dict(type='RescaleToZeroOne', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Normalize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        to_rgb=False,
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]),
    dict(type='ImageToTensor', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Collect',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        meta_keys=[f'img_{domain_a}_path', f'img_{domain_b}_path'])
]

test_pipeline = [
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_a}',
        flag='color'),
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_b}',
        flag='color'),
    dict(
        type='Resize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        scale=(512, 512),
        interpolation='bicubic'),
    dict(type='RescaleToZeroOne', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Normalize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        to_rgb=False,
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]),
    dict(type='ImageToTensor', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Collect',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        meta_keys=[f'img_{domain_a}_path', f'img_{domain_b}_path'])
]

data = dict(
    train=dict(
        dataroot=dataroot,
        pipeline=train_pipeline,
        domain_a=domain_a,
        domain_b=domain_b),
    val=dict(
        dataroot=dataroot,
        domain_a=domain_a,
        domain_b=domain_b,
        pipeline=test_pipeline),
    test=dict(
        dataroot=dataroot,
        domain_a=domain_a,
        domain_b=domain_b,
        pipeline=test_pipeline))

optimizer = dict(
    generators=dict(type='Adam', lr=0.0002, betas=(0.5, 0.999)),
    discriminators=dict(type='Adam', lr=0.0002, betas=(0.5, 0.999)))

# learning policy
lr_config = dict(
    policy='Linear', by_epoch=False, target_lr=0, start=125000, interval=1250)

checkpoint_config = dict(interval=10000, save_optimizer=True, by_epoch=False)
custom_hooks = [
    dict(
        type='MMGenVisualizationHook',
        output_dir='training_samples',
        res_name_list=[f'fake_{domain_a}', f'fake_{domain_b}'],
        interval=5000)
]

runner = None
use_ddp_wrapper = True
total_iters = 250000
# load_from = './work_dirs/cyclegan_blackclover/ckpt/cyclegan_blackclover/iter_150000.pth'
resume_from = './work_dirs/cyclegan_blackclover/ckpt/cyclegan_blackclover/iter_150000.pth'
workflow = [('train', 1)]
exp_name = 'cyclegan_blackclover'
work_dir = f'./work_dirs/{exp_name}'
# testA: 126, testB:126
num_images = 126
metrics = dict(
    FID=dict(type='FID', num_images=num_images, image_shape=(3, 512, 512)),
    IS=dict(
        type='IS',
        num_images=num_images,
        image_shape=(3, 512, 512),
        inception_args=dict(type='pytorch')))

evaluation = dict(
    type='TranslationEvalHook',
    target_domain=domain_b,
    interval=10000,
    metrics=[
        dict(type='FID', num_images=num_images, bgr2rgb=True),
        dict(
            type='IS',
            num_images=num_images,
            inception_args=dict(type='pytorch'))
    ],
    best_metric=['fid', 'is'])

Overwriting configs/cyclegan/cyclegan_lsgan_id0_resnet_in_summer2winter_b1x1_250k.py


In [20]:
!python tools/train.py configs/cyclegan/cyclegan_lsgan_id0_resnet_in_summer2winter_b1x1_250k.py --work-dir ./work_dirs/cyclegan_blackclover --no-validate

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
tools/train.py:98: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
tools/train.py:108: UserWarning: Setting MKL_NUM_THREADS environ